## Coursera_IBM_Applied-Data-Science-Capstone
#### This Notebook represents my work for the Coursera_IBM_Applied Data Science Capstone as one of the various courses of IBM Data Science Professional Certificate

##### Ahmed Tealeb

# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Business Problem section 

#### Background

According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales. 

#### Business Problem

In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario? 

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.




##  Data section

Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.


## Methodology section

The Methodology section will describe the main components of our analysis and predication system.
The Methodology section comprises four stages: 
    1. Collect Inspection Data
    2. Explore and Understand Data
    3. Data Preparation and Preprocessing 
    4. Modeling

#### 1. Collect Inspection Data

After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [76]:
# importing the Libraries
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [78]:
# Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.

#### 2. Explore and Understand Data

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows: 

In [79]:
df_ppd.head(5) 

,{68FEB20B-EAA3-38DA-E053-6C04A8C051AE},320000,2018-02-14 00:00,CO10 1LG,S,N,F,9,Unnamed: 8,ST BARTHOLOMEWS LANE,Unnamed: 10,SUDBURY,BABERGH,SUFFOLK,A,A.1
0,{68FEB20B-EAA4-38DA-E053-6C04A8C051AE},120000,2018-02-02 00:00,NR33 8FN,F,N,L,39,NaN,BUTTERMERE WAY,CARLTON COLVILLE,LOWESTOFT,WAVENEY,SUFFOLK,A,A
1,{68FEB20B-EAA5-38DA-E053-6C04A8C051AE},385000,2018-03-01 00:00,IP13 9JY,S,N,F,1,NaN,MILLERS WAY,FRAMLINGHAM,WOODBRIDGE,SUFFOLK COASTAL,SUFFOLK,A,A
2,{68FEB20B-EAA6-38DA-E053-6C04A8C051AE},265000,2018-03-07 00:00,IP7 5AR,D,N,F,"MYRTLE COTTAGE, 17A",NaN,BENTON STREET,HADLEIGH,IPSWICH,BABERGH,SUFFOLK,A,A
3,{68FEB20B-EAA7-38DA-E053-6C04A8C051AE},166000,2018-03-05 00:00,IP3 9NR,S,N,F,13,NaN,CORTON ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
4,{68FEB20B-EAA8-38DA-E053-6C04A8C051AE},260000,2018-03-09 00:00,IP12 4BF,F,N,L,CLARKSON COURT,42,IPSWICH ROAD,NaN,WOODBRIDGE,SUFFOLK COASTAL,SUFFOLK,A,A


In [80]:
df_ppd.shape

(821379, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly. 

#### 3. Data Preparation and Preprocessing

At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps: 
- Rename the column names 
- Format the date column 
- Sort data by date of sale 
- Select data only for the city of London 
- Make a list of street names in London 
- Calculate the street-wise average price of the property
- Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names 
- Join the data to find the coordinates of locations which fit into client's budget
- Plot recommended locations on London map along with current market prices






In [81]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [82]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [83]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [84]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [85]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [86]:
# Display the dataframe
df_affordable

,Street,Avg_Price
185,ALBION SQUARE,2.450000e+06
372,ANHALT ROAD,2.435000e+06
386,ANSDELL TERRACE,2.250000e+06
399,APPLEGARTH ROAD,2.400000e+06
651,AYLESTONE AVENUE,2.286667e+06
796,BARONSMEAD ROAD,2.375000e+06
914,BEAUCLERC ROAD,2.480000e+06
1136,BICKENHALL STREET,2.387143e+06
1155,BILLING ROAD,2.200000e+06
1172,BIRCHLANDS AVENUE,2.217000e+06


In [87]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [88]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 185
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 185, dtype: object
item.Street only: ALBION SQUARE
index: 372
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 372, dtype: object
item.Street only: ANHALT ROAD
index: 386
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 386, dtype: object
item.Street only: ANSDELL TERRACE
index: 399
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 399, dtype: object
item.Street only: APPLEGARTH ROAD
index: 651
item: Street       AYLESTONE AVENUE
Avg_Price         2.28667e+06
Name: 651, dtype: object
item.Street only: AYLESTONE AVENUE
index: 796
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 796, dtype: object
item.Street only: BARONSMEAD ROAD
index: 914
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 914, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1136
item: Street       BICKENHALL STREET
Avg_Price          2.38714e+06


In [89]:
geolocator = Nominatim()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [91]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [92]:
df_affordable

,Street,Avg_Price,city_coord
185,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
372,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
386,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
399,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
651,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)"
796,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
914,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)"
1136,BICKENHALL STREET,2.387143e+06,"(51.5211969, -0.1589341)"
1155,BILLING ROAD,2.200000e+06,"(51.4818833, -0.1878624)"
1172,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"


In [93]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

In [94]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
185,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
372,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
386,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
399,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
651,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)",51.540916,-0.217874
796,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
914,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)",51.499577,-0.229033
1136,BICKENHALL STREET,2.387143e+06,"(51.5211969, -0.1589341)",51.521197,-0.158934
1155,BILLING ROAD,2.200000e+06,"(51.4818833, -0.1878624)",51.481883,-0.187862
1172,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468


In [95]:
df = df_affordable.drop(columns=['city_coord'])


In [96]:
df

,Street,Avg_Price,Latitude,Longitude
185,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
372,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
386,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
399,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
651,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874
796,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
914,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033
1136,BICKENHALL STREET,2.387143e+06,51.521197,-0.158934
1155,BILLING ROAD,2.200000e+06,51.481883,-0.187862
1172,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468


In [97]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [98]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [99]:
#Define Foursquare Credentials and Version

CLIENT_ID = '1DSRC5HBGQXKI2PVYIUZY2EDC2N0QXKLJ32YSJVNLXJNXP12' # Foursquare ID
CLIENT_SECRET = 'M551JVRZT2OYTPWAGQQTFDZF4UBOHDQTRI4ACE1Y0T14SVIF' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1DSRC5HBGQXKI2PVYIUZY2EDC2N0QXKLJ32YSJVNLXJNXP12
CLIENT_SECRET:M551JVRZT2OYTPWAGQQTFDZF4UBOHDQTRI4ACE1Y0T14SVIF


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

#### 4. Modeling

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
AYLESTONE AVENUE
BARONSMEAD ROAD
BEAUCLERC ROAD
BICKENHALL STREET
BILLING ROAD
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BRIXTON ROAD
BROADLANDS ROAD
BUCKINGHAM PALACE ROAD
BURBAGE ROAD
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANFIELD GARDENS
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHISWICK MALL
CLARENDON STREET
CLEVELAND SQUARE
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRAVEN STREET
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIBSON SQUARE
GIRDLERS ROAD
GORDON PLACE
GRAHAM TERRACE
HARMAN DRIVE
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGH STREET COLLIERS WOOD
HIGHGATE HIGH STREET
HILLGATE PLACE
HOLLYWOOD MEWS
HONEYWELL ROAD
HUNTER ROAD
JOHN ISL

In [102]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
3,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
4,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
5,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
6,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
7,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket
8,ALBION SQUARE,-41.273758,173.289393,Hopgood's,-41.274749,173.283831,Restaurant
9,ALBION SQUARE,-41.273758,173.289393,Sprig & Fern,-41.274508,173.286527,Brewery


In [103]:
location_venues.groupby('Street').count()


,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,27,27,27,27,27,27
ANHALT ROAD,15,15,15,15,15,15
ANSDELL TERRACE,56,56,56,56,56,56
APPLEGARTH ROAD,4,4,4,4,4,4
AYLESTONE AVENUE,5,5,5,5,5,5
BARONSMEAD ROAD,13,13,13,13,13,13
BEAUCLERC ROAD,33,33,33,33,33,33
BICKENHALL STREET,93,93,93,93,93,93
BILLING ROAD,57,57,57,57,57,57


In [104]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))


There are 322 uniques categories.


In [105]:
location_venues.shape

(5107, 7)

In [106]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,ALBION SQUARE,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.037037,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000
1,ANHALT ROAD,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000
2,ANSDELL TERRACE,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.017857,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.017857,0.000000
3,APPLEGARTH ROAD,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000
4,AYLESTONE AVENUE,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000
5,BARONSMEAD ROAD,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000
6,BEAUCLERC ROAD,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000
7,BICKENHALL STREET,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.010753,0.000000,0.00,0.000000,0.010753
8,BILLING ROAD,0.00,0.000000,0.000000,0.00,0.000000,0.017544,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000
9,BIRCHLANDS AVENUE,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000


In [108]:
london_grouped.shape


(127, 323)

In [109]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.22
1                Bar  0.07
2                Pub  0.07
3         Restaurant  0.07
4  Indian Restaurant  0.07


----ANHALT ROAD----
                 venue  freq
0                  Pub  0.27
1  Japanese Restaurant  0.07
2        Grocery Store  0.07
3       Farmers Market  0.07
4               Garden  0.07


----ANSDELL TERRACE----
                venue  freq
0  Italian Restaurant  0.07
1                Café  0.07
2      Clothing Store  0.07
3                 Pub  0.05
4   Indian Restaurant  0.05


----APPLEGARTH ROAD----
                  venue  freq
0                   Pub  0.50
1             Nightclub  0.25
2                Casino  0.25
3  Outdoor Supply Store  0.00
4      Pedestrian Plaza  0.00


----AYLESTONE AVENUE----
               venue  freq
0               Park   0.4
1           Bus Stop   0.2
2               Café   0.2
3      Movie Theater   0.2
4  Accessories Store   0.0


----BARONSMEAD ROAD----
     

In [110]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))



In [112]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)



In [113]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Coffee Shop,Bar,Pub,Indian Restaurant,Restaurant,New American Restaurant,Park,Supermarket,Beer Garden
1,ANHALT ROAD,Pub,Pizza Place,Garden,Farmers Market,English Restaurant,Grocery Store,Gym / Fitness Center,Diner,Cocktail Bar,French Restaurant
2,ANSDELL TERRACE,Clothing Store,Café,Italian Restaurant,Hotel,Indian Restaurant,Restaurant,Pub,Garden,Bakery,Juice Bar
3,APPLEGARTH ROAD,Pub,Nightclub,Casino,Fish Market,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
4,AYLESTONE AVENUE,Park,Bus Stop,Movie Theater,Café,Yoga Studio,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [114]:
venues_sorted.shape

(127, 11)

In [115]:
london_grouped.shape

(127, 323)

In [116]:
london_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby. 

In [117]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 1, 4, 3, 0, 3, 1, 3, 2, 2, 1, 3, 0, 3, 4, 1, 3, 3, 1, 4, 1, 2,
       1, 0, 0, 3, 1, 1, 4, 0, 4, 1, 3, 3, 2, 3, 3, 3, 3, 1, 3, 2, 3, 0,
       4, 2, 2, 0, 4, 4], dtype=int32)

In [118]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
185,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
372,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
386,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
399,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
651,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874


In [119]:
london_grouped_clustering.shape

(136, 4)

In [120]:
df.shape

(136, 4)

In [121]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [122]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [123]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
185,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Coffee Shop,Bar,Pub,Indian Restaurant,Restaurant,New American Restaurant,Park,Supermarket,Beer Garden
372,ANHALT ROAD,2.435000e+06,51.480326,-0.166761,1,Pub,Pizza Place,Garden,Farmers Market,English Restaurant,Grocery Store,Gym / Fitness Center,Diner,Cocktail Bar,French Restaurant
386,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,4,Clothing Store,Café,Italian Restaurant,Hotel,Indian Restaurant,Restaurant,Pub,Garden,Bakery,Juice Bar
399,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,3,Pub,Nightclub,Casino,Fish Market,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
651,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874,0,Park,Bus Stop,Movie Theater,Café,Yoga Studio,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
796,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,3,Restaurant,Community Center,Park,Nature Preserve,Pub,Coffee Shop,Food & Drink Shop,Pizza Place,Café,Bookstore
914,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,1,Hotel,Coffee Shop,Pub,Grocery Store,Thai Restaurant,Gastropub,Fast Food Restaurant,Supermarket,Bus Stop,Bed & Breakfast
1136,BICKENHALL STREET,2.387143e+06,51.521197,-0.158934,3,Hotel,Chinese Restaurant,Restaurant,Italian Restaurant,Gastropub,Café,Pub,Pizza Place,Movie Theater,Sandwich Place
1155,BILLING ROAD,2.200000e+06,51.481883,-0.187862,2,Pub,Italian Restaurant,Soccer Stadium,Grocery Store,Gastropub,Restaurant,Café,Park,Sandwich Place,Sports Bar
1172,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,2,Pub,Lake,Breakfast Spot,Chinese Restaurant,Bakery,Train Station,Coffee Shop,French Restaurant,Department Store,Design Studio


In [129]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [130]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
651,2.286667e+06,Park,Bus Stop,Movie Theater,Café,Yoga Studio,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1571,2.329904e+06,Caribbean Restaurant,Pizza Place,Coffee Shop,Pub,Tapas Restaurant,Nightclub,Mexican Restaurant,Market,Cupcake Shop,Brazilian Restaurant
2073,2.300000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2219,2.286679e+06,Café,Bar,Italian Restaurant,Pub,Pizza Place,Coffee Shop,Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Yoga Studio
2484,2.287500e+06,Pub,Gift Shop,Gym / Fitness Center,Brewery,Art Museum,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [131]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
185,2.450000e+06,Café,Coffee Shop,Bar,Pub,Indian Restaurant,Restaurant,New American Restaurant,Park,Supermarket,Beer Garden
372,2.435000e+06,Pub,Pizza Place,Garden,Farmers Market,English Restaurant,Grocery Store,Gym / Fitness Center,Diner,Cocktail Bar,French Restaurant
914,2.480000e+06,Hotel,Coffee Shop,Pub,Grocery Store,Thai Restaurant,Gastropub,Fast Food Restaurant,Supermarket,Bus Stop,Bed & Breakfast
1445,2.475833e+06,Bar,Lake,Middle Eastern Restaurant,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
1775,2.445000e+06,Café,Brewery,Bakery,Coffee Shop,Gastropub,Athletics & Sports,Diner,Stadium,Gas Station,Garden Center


In [132]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1155,2200000.0,Pub,Italian Restaurant,Soccer Stadium,Grocery Store,Gastropub,Restaurant,Café,Park,Sandwich Place,Sports Bar
1172,2217000.0,Pub,Lake,Breakfast Spot,Chinese Restaurant,Bakery,Train Station,Coffee Shop,French Restaurant,Department Store,Design Studio
2058,2200000.0,Nightlife Spot,Playground,Soccer Field,Yoga Studio,Fish Market,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2773,2225000.0,College Hockey Rink,College Cafeteria,Bar,Deli / Bodega,Diner,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
3686,2225000.0,Café,Pub,Italian Restaurant,Bakery,Museum,Thai Restaurant,Coffee Shop,Bookstore,Monument / Landmark,French Restaurant


In [133]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
399,2.400000e+06,Pub,Nightclub,Casino,Fish Market,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
796,2.375000e+06,Restaurant,Community Center,Park,Nature Preserve,Pub,Coffee Shop,Food & Drink Shop,Pizza Place,Café,Bookstore
1136,2.387143e+06,Hotel,Chinese Restaurant,Restaurant,Italian Restaurant,Gastropub,Café,Pub,Pizza Place,Movie Theater,Sandwich Place
1517,2.397132e+06,Chinese Restaurant,Grocery Store,Coffee Shop,Seafood Restaurant,Fast Food Restaurant,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
1586,2.347500e+06,Pub,Grocery Store,IT Services,Garden Center,Home Service,Yoga Studio,Fish Market,Exhibit,Falafel Restaurant,Farm


In [134]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
386,2.250000e+06,Clothing Store,Café,Italian Restaurant,Hotel,Indian Restaurant,Restaurant,Pub,Garden,Bakery,Juice Bar
1748,2.259052e+06,Hotel,Historic Site,Coffee Shop,Palace,Pub,Indian Restaurant,Supermarket,Gift Shop,Memorial Site,Garden
1992,2.278000e+06,Coffee Shop,Café,Pizza Place,Italian Restaurant,Bookstore,Grocery Store,Hotel,Asian Restaurant,Dessert Shop,Furniture / Home Store
2437,2.250000e+06,Gastropub,Yoga Studio,Flea Market,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
2602,2.250000e+06,Spa,American Restaurant,Gym,Italian Restaurant,Women's Store,Hotel,Seafood Restaurant,Sandwich Place,Gym / Fitness Center,Cosmetics Shop


## Results and Discussion section

First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives. 

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer. 


## Conclusion

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario? 

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments. 

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer. 

### Thank you for completing this notebook!

This notebook was created by [Ahmed Tealeb](https://www.linkedin.com/in/ahmedtealeb/).

This notebook is part of an assignment on **Coursera** called *Applied Data Science Capstone*. 